In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import os
import shutil
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.models import load_model

In [2]:
path  = "C:\\Users\\16395\\Desktop\\Sorghom\\"

In [3]:
train_df = pd.read_csv(path +'train_cultivar_mapping.csv')
trpath = "C:\\Users\\16395\\Desktop\\Sorghom\\"
train_df.dropna()

,image,cultivar
0,2017-06-16__12-24-20-930.png,PI_257599
1,2017-06-02__16-48-57-866.png,PI_154987
2,2017-06-12__13-18-07-707.png,PI_92270
3,2017-06-22__13-18-06-841.png,PI_152651
4,2017-06-26__12-56-48-642.png,PI_176766
...,...,...
22189,2017-06-16__12-27-16-266.png,PI_170787
22190,2017-06-28__11-19-57-454.png,PI_156393
22191,2017-06-28__10-20-32-417.png,PI_152923
22192,2017-06-28__12-47-02-714.png,PI_257599


In [4]:
IMG_SIZE = 256
BATCH_SIZE = 8

In [11]:
train_df['cultivar']=train_df['cultivar'].astype(str)

In [6]:
# Set paths and split ratio
data_path = "C:\\Users\\16395\\Desktop\\Sorghom\\"
csv_path =  "C:\\Users\\16395\\Desktop\\Sorghom\\train_cultivar_mapping.csv"
split_ratio = 0.2

# Load csv file
df = pd.read_csv(csv_path)

# Split data into training and validation sets
train_df, val_df = train_test_split(df, test_size=split_ratio, random_state=42)


In [7]:
train_df['cultivar']=train_df['cultivar'].astype(str)
val_df['cultivar']=val_df['cultivar'].astype(str)

In [9]:
train_df.count()
data_path = "C:\\Users\\16395\\Desktop\\Sorghom\\train_images"
csv_path =  "C:\\Users\\16395\\Desktop\\Sorghom\\train_cultivar_mapping.csv"
split_ratio = 0.2

# Load csv file
df = pd.read_csv(csv_path)

# Split data into training and validation sets
train_df, val_df = train_test_split(df, test_size=split_ratio, random_state=42)


In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=path + 'train_images',
    x_col='image',
    y_col='cultivar',
    class_mode='sparse',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

valid_generator = train_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=path + 'train_images',
    x_col='image',
    y_col='cultivar',
    class_mode='sparse',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

Found 17754 validated image filenames belonging to 100 classes.


C:\Users\16395\anaconda3\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


Found 4439 validated image filenames belonging to 100 classes.


In [13]:
base_model = keras.applications.EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [14]:
base_model.layers

In [15]:
new_model= base_model.layers[0:430]

In [16]:
print(len(base_model.layers))

475


In [17]:
new_input=base_model.input
new_output=new_model[-1].output
new_model=keras.Model(new_input,new_output)

In [18]:


inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
output = new_model(inputs)
output = GlobalAveragePooling2D()(output)
output = Dense(100, activation='softmax')(output)

new_model = keras.Model(inputs, output)

new_model.summary()

new_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 model (Functional)          (None, 8, 8, 1632)        11652263  
                                                                 
 global_average_pooling2d (G  (None, 1632)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 100)               163300    
                                                                 
Total params: 11,815,563
Trainable params: 11,720,084
Non-trainable params: 95,479
_________________________________________________________________


In [19]:
history = new_model.fit(train_generator, validation_data=valid_generator, epochs=10)

Epoch 1/10
2220/2220 [==============================] - 2480s 1s/step - loss: 3.5456 - accuracy: 0.2641 - val_loss: 2.6695 - val_accuracy: 0.3501
Epoch 2/10
2220/2220 [==============================] - 1052s 474ms/step - loss: 1.7111 - accuracy: 0.5282 - val_loss: 2.0605 - val_accuracy: 0.4940
Epoch 3/10
2220/2220 [==============================] - 908s 409ms/step - loss: 1.3798 - accuracy: 0.6066 - val_loss: 1.2904 - val_accuracy: 0.6303
Epoch 4/10
2220/2220 [==============================] - 883s 398ms/step - loss: 1.1987 - accuracy: 0.6503 - val_loss: 1.1996 - val_accuracy: 0.6634
Epoch 5/10
2220/2220 [==============================] - 877s 395ms/step - loss: 1.0707 - accuracy: 0.6853 - val_loss: 0.9239 - val_accuracy: 0.7385
Epoch 6/10
2220/2220 [==============================] - 870s 392ms/step - loss: 0.9658 - accuracy: 0.7174 - val_loss: 1.0023 - val_accuracy: 0.7207
Epoch 7/10
2220/2220 [==============================] - 875s 394ms/step - loss: 0.8866 - accuracy: 0.7366 - val_l

In [20]:
new_model.fit(train_generator, validation_data=valid_generator, epochs=5)

Epoch 1/5
2220/2220 [==============================] - 5175s 2s/step - loss: 0.6703 - accuracy: 0.8027 - val_loss: 0.6522 - val_accuracy: 0.8209
Epoch 2/5
2220/2220 [==============================] - 2651s 1s/step - loss: 0.6367 - accuracy: 0.8114 - val_loss: 0.6042 - val_accuracy: 0.8340
Epoch 3/5
2220/2220 [==============================] - 15279s 7s/step - loss: 0.5995 - accuracy: 0.8205 - val_loss: 0.8697 - val_accuracy: 0.7815
Epoch 4/5
2220/2220 [==============================] - 5181s 2s/step - loss: 0.5685 - accuracy: 0.8325 - val_loss: 1.2493 - val_accuracy: 0.7202
Epoch 5/5
2220/2220 [==============================] - 832s 375ms/step - loss: 0.5447 - accuracy: 0.8385 - val_loss: 0.7194 - val_accuracy: 0.8148
